In [1]:
import json
import re
import requests

import nltk
nltk.download('stopwords')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing  import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import f1_score
from IPython.core.display import HTML, display

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andrey\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train_df = pd.read_csv("train.csv")

train_df.head()

,id,url,title,target
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,3,colorbox.spb.ru,Не Беси Меня Картинки,False
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False


In [3]:
test_df = pd.read_csv("test.csv")

test_df.head()

,id,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67


In [4]:
train_df["target"].value_counts()

False    118594
True      16715
Name: target, dtype: int64

In [5]:
X_train = train_df["title"].values
X_test = test_df["title"].values
y_train = train_df["target"].astype(int).values

In [6]:
url0 = train_df[train_df.target == 0].url.str.count('xxx')
url1 = train_df[train_df.target == 1].url.str.count('xxx')


title0 = train_df[train_df.target == 0].title.str.count('xxx')
title1 = train_df[train_df.target == 1].title.str.count('xxx')

In [7]:
url1.value_counts()

0    16005
1      709
2        1
Name: url, dtype: int64

In [8]:
url0.value_counts()

0    118593
1         1
Name: url, dtype: int64

In [9]:
title0.value_counts()

0    118590
1         3
2         1
Name: title, dtype: int64

In [10]:
title1.value_counts()

0    16191
1      467
2       53
4        3
3        1
Name: title, dtype: int64

In [11]:
train_df['url_xxx'] = train_df.url.str.count('xxx')
train_df['title_xxx'] = train_df.title.str.count('xxx')

In [12]:
train_df.head()

,id,url,title,target,url_xxx,title_xxx
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False,0,0
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False,0,0
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False,0,0
3,3,colorbox.spb.ru,Не Беси Меня Картинки,False,0,0
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False,0,0


In [13]:
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-zА-ЯЁа-яё0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "")
    df[text_field] = df[text_field].str.replace(r"www.", "")
    df[text_field] = df[text_field].str.replace(r"m.", "")
    df[text_field] = df[text_field].str.replace(r".com", "")
    df[text_field] = df[text_field].str.replace(r".net", "")
    df[text_field] = df[text_field].str.replace(r".ru", "")
    df[text_field] = df[text_field].str.replace(r".by", "")
    df[text_field] = df[text_field].str.replace(r".tv", "")
    df[text_field] = df[text_field].str.lower()
    return df

train_df = standardize_text(train_df, "url")
train_df = standardize_text(train_df, "title")

train_df.head()

,id,url,title,target,url_xxx,title_xxx
0,0,kp,"экс министр экономики молдовы главе мидэи, ц...",False,0,0
1,1,kp,эта песня стала известна многим телезрителям б...,False,0,0
2,2,fanserials,банши 4 сезон 2 серия бремя красоты смотреть о...,False,0,0
3,3,colorbox spb,не беси меня картинки,False,0,0
4,4,tula sport,в новомосковске сыграют следж хоккеисты алекси...,False,0,0


In [110]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

train_df["tokens_url"] = train_df["url"].apply(tokenizer.tokenize)
train_df["tokens_title"] = train_df["title"].apply(tokenizer.tokenize)
train_df.head()

,id,url,title,target,url_xxx,title_xxx,tokens_url,tokens_title
0,0,kp,"экс министр экономики молдовы главе мидэи, ц...",False,0,0,[kp],"[экс, министр, экономики, молдовы, главе, мидэ..."
1,1,kp,эта песня стала известна многим телезрителям б...,False,0,0,[kp],"[эта, песня, стала, известна, многим, телезрит..."
2,2,fanserials,банши 4 сезон 2 серия бремя красоты смотреть о...,False,0,0,[fanserials],"[банши, 4, сезон, 2, серия, бремя, красоты, см..."
3,3,colorbox spb,не беси меня картинки,False,0,0,"[colorbox, spb]","[не, беси, меня, картинки]"
4,4,tula sport,в новомосковске сыграют следж хоккеисты алекси...,False,0,0,"[tula, sport]","[в, новомосковске, сыграют, следж, хоккеисты, ..."


In [71]:
def cv(data):
    count_vectorizer = CountVectorizer()

    emb = count_vectorizer.fit_transform(data)

    return emb, count_vectorizer

list_corpus = train_df["title"].tolist()
list_labels = train_df["target"].tolist()

X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, test_size=0.2, 
                                                                                random_state=40)

X_train_counts, count_vectorizer = cv(X_train)
X_test_counts = count_vectorizer.transform(X_test)

In [73]:
clf = LogisticRegression(C=30.0, class_weight='balanced', solver='newton-cg', 
                         multi_class='multinomial', n_jobs=-1, random_state=40)
clf.fit(X_train_counts, y_train)

y_predicted_counts = clf.predict(X_test_counts)

In [75]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

def get_metrics(y_test, y_predicted):  
    # true positives / (true positives+false positives)
    precision = precision_score(y_test, y_predicted, pos_label=None,
                                    average='weighted')             
    # true positives / (true positives + false negatives)
    recall = recall_score(y_test, y_predicted, pos_label=None,
                              average='weighted')
    
    # harmonic mean of precision and recall
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')
    
    # true positives + true negatives/ total
    accuracy = accuracy_score(y_test, y_predicted)
    return accuracy, precision, recall, f1

accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.986, precision = 0.986, recall = 0.986, f1 = 0.986


### Submit

In [14]:
test_df = standardize_text(test_df, "url")
test_df = standardize_text(test_df, "title")

In [15]:
test_df

,id,url,title
0,135309,koersant,шестой кассационный суд в самаре начнет работу...
1,135310,urexpert online,"что такое индексация алиментов, кем и в каких ..."
2,135311,ieriha,женщинам империя меха part 12
3,135312,national porn,"небритые, волосатые киски порно всех стран и ..."
4,135313,2gis,67
...,...,...,...
165373,300682,etp arek,arek запчасти для грузовых и легковых автомо...
165374,300683,il,"лилия якупова караганда, карагандинская обла..."
165375,300684,xn 8sbnqchpeeeth xn p1ai,администрация лесного района тверской области ...
165376,300685,sunho cdn aproject org,сонник изменение сознания к чему снится измен...


In [134]:
X_test = test_df['title'].tolist()

In [125]:
test_df.shape

(165378, 4)

In [112]:
def cv(data):
    count_vectorizer = CountVectorizer()

    emb = count_vectorizer.fit_transform(data)

    return emb, count_vectorizer

X_train = train_df["title"].tolist()
y_train = train_df["target"].tolist()

X_train_counts, count_vectorizer = cv(X_train)
X_test_counts = count_vectorizer.transform(X_test)

In [113]:
clf = LogisticRegression(C=30.0, class_weight='balanced', solver='newton-cg', 
                         multi_class='multinomial', n_jobs=-1, random_state=40)
clf.fit(X_train_counts, y_train)

LogisticRegression(C=30.0, class_weight='balanced', multi_class='multinomial',
                   n_jobs=-1, random_state=40, solver='newton-cg')

In [114]:
y_predicted_counts = clf.predict(X_test_counts)

In [119]:
y_predicted_counts

array([False, False, False, ..., False, False, False])

In [120]:
test_df["target"] = y_predicted_counts.astype(bool)

test_df[["id", "target"]].to_csv("ml_baseline.csv", index=False)

!cat ml_baseline.csv | head

"cat" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [121]:
test_df

,id,url,title,target
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...,False
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ...",False
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12,False
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ...",True
4,135313,2gis.ru,67,True
...,...,...,...,...
165373,300682,etp.armtek.ru,Armtek - запчасти для грузовых и легковых авто...,False
165374,300683,mail.ru,"Лилия Якупова - Караганда, Карагандинская обла...",False
165375,300684,xn----8sbnqchpeeeth.xn--p1ai,Администрация Лесного района Тверской области ...,False
165376,300685,www-sunhome-ru.cdn.ampproject.org,Сонник Изменение сознания. К чему снится Измен...,False


In [122]:
test_df['target'].value_counts()

False    144893
True      20485
Name: target, dtype: int64

TF_IDF

In [135]:
X_train = train_df["title"].tolist()
y_train = train_df["target"].tolist()

In [136]:
def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer()

    train = tfidf_vectorizer.fit_transform(data)

    return train, tfidf_vectorizer

X_train_tfidf, tfidf_vectorizer = tfidf(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [137]:
clf_tfidf = LogisticRegression(C=30.0, class_weight='balanced', solver='newton-cg', 
                         multi_class='multinomial', n_jobs=-1, random_state=40)
clf_tfidf.fit(X_train_tfidf, y_train)

y_predicted_tfidf = clf_tfidf.predict(X_test_tfidf)

In [138]:
test_df["target"] = y_predicted_tfidf.astype(bool)

test_df[["id", "target"]].to_csv("ml_baseline.csv", index=False)

!cat ml_baseline.csv | head

"cat" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [144]:
test_df[370:390]

,id,url,title,target
370,135679,vsedoor,входная дверь сенатор 2к (венге),False
371,135680,tutu,tutu выписка билета,False
372,135681,stavropol cian,76 объявлений снять квартиру недорого в пяти...,False
373,135682,sntch,предполагаемая дочь путина катерина тихонова р...,False
374,135683,gigabaza,установление причастности тлеющего табачного и...,False
375,135684,worldoftanks eu,"world of tanks, mmo free to play de acci n sob...",False
376,135685,erosweet,сексапильная молодуха широко раскрыла свой вол...,True
377,135686,rtrik,"пальто юрс, красный (модель 18 872 5) белору...",False
378,135687,newbalance rket,кроссовки мужские кожа черный зимние в москве,False
379,135688,il,маски хамелеон 1 млн результатов поиск mail ru,False


In [140]:
test_df['target'].value_counts()

False    144966
True      20412
Name: target, dtype: int64